# AWS Deployment Guide
> This notebook will guide you step-by-step on how to configure your AWS instance and docker container in order to deploy your backend. Thank you to Samit Poojary, Ethan Tran, Edwin Abraham, Mr. Mortensen and Jeffrey Fonseca for resources and instructions.

- toc: true
- badges: false
- author: Ethan Tran, Samit Poojary, and Azeem Khan
- permalink: /week22/deploymentGuide
- categories: [week 22]

## Accessing AWS

### Server Setup and Initial Deployment
> Development Operations (DevOps) begins with server setup.    
### Amazon Web Services (AWS): Electric Cloud Compute (EC2) Setup
- To begin, head to  the "Instances" dropdown on AWS and select "Instances."

![image1](https://user-images.githubusercontent.com/109186517/220186297-adf65729-a8db-4506-ae03-cd75273d92d0.png)

- From here, a variety of instances will show up. For this project, depending on which teacher you have, select either "NCS.cf Yeung CSP" or "NCS.gq Mort CSP"

![image2](https://user-images.githubusercontent.com/109186517/220186519-ca92be0c-c9e6-4173-bbdb-9715a13a8a69.png)


## Local Setup

### Finding a Port

In AWS;

1. Run `docker ps` or `sudo docker ps` and find a port number starting in 8--- that is not in use. (I will be using 8081)


### Setting up Docker

2. Open VSCode and navigate to your Flask repository (backend)

3. Make sure your `Dockerfile` matches the template below.

In [ ]:
FROM docker.io/python:3.10

WORKDIR /

# --- [Install python and pip] ---
RUN apt-get update && apt-get upgrade -y && \
    apt-get install -y python3 python3-pip git
COPY . /app

RUN pip install --no-cache-dir -r requirements.txt
RUN pip install gunicorn

ENV GUNICORN_CMD_ARGS="--workers=3 --bind=0.0.0.0:8080"

EXPOSE 8080

CMD [ "gunicorn", "main:app" ]

4. Edit your `docker-compose.yml` according to the template below.

In [ ]:
version: '3'
services:
        web:
                image: flask_port_v1 # Change the image name to something unique
                build: .
                ports:
                        - "8---:8080" # Edit the number on the left to match the port you chose
                volumes:
                        - ./volumes:/volumes
                        - ./instance:/instance
                restart: unless-stopped


5. Test `docker-compose up` or `sudo docker-compose up` in your VSCode terminal (don't forget to 'cd' into the right repo.)

6. After it's done building, type in `http://localhost:8---` in your browser (replace '8---' with your port number you've chosen)

7. If all runs smoothly, push your changes to Github and continue to [AWS setup](https://azeem-khan1.github.io/fastpages-project/week22/deploymentGuide#AWS-Setup)

## AWS Setup

In the AWS terminal;

1. `cd ~`

2. Clone your backend repo: `git clone github.com/server/project.git my-unique-name`

3. Navigate to your repo: `cd my-unique-name`

4. Install your requirements: `pip install -r requirements.txt`

6. Build your site: `docker-compose up -d --build`

7. Test your site: `curl localhost:8---` (replace '8---' with your port number)
> This should show you all the html content of your site. Double check some things that are unique to your site to ensure your port is accurate and working.

If all runs smooth, continue to [DNS & NGINX Setup](https://azeem-khan1.github.io/fastpages-project/week22/deploymentGuide#DNS-&-NGINX-Setup)


## DNS & NGINX Setup

> Follow [Jeffrey's guide to duckdns](https://moonpiedumplings.github.io/quartotest/posts/duckdns/) (public IP can be found below the terminal)


### Nginx setup in AWS terminal

1. Navigate to nginx: `cd /etc/nginx/sites-available`

2. Create an nginx config file: `sudo nano yourUniqueName`

3. Use the format below to write into your config file

In [ ]:
server {
   listen 80;
    listen [::]:80;
    server_name -----.duckdns.org; # CHANGE SERVER NAME TO YOUR REGISTERED DOMAIN
    location / {
        proxy_pass http://localhost:8---; # CHANGE PORT TO YOUR UNIQUE PORT
        # Simple requests
        if ($request_method ~* “(GET|POST|PUT|DELETE)“) { # Customize Request methods based on your needs
                add_header “Access-Control-Allow-Origin”  *;
        }
        # Preflighted requests
        if ($request_method = OPTIONS ) {
                add_header “Access-Control-Allow-Origin”  *;
                add_header “Access-Control-Allow-Methods” “GET, POST, PUT, DELETE, OPTIONS, HEAD”; # Make sure the request methods above match here
                add_header “Access-Control-Allow-Headers” “Authorization, Origin, X-Requested-With, Content-Type, Accept”;
                return 200;
        }
    }
}

4. To save changes, `ctl X` or `cmd X`, then `y`, then `enter`

5. Create a symbolic link: `cd /etc/nginx/sites-enabled`, then `ln -s /etc/nginx/sites-available/yourUniqueName /etc/nginx/sites-enabled` (change yourUniqueName to your nginx config file name)

6. Validate by running: `sudo nginx -t`

7. Restart nginx by running `sudo systemctl restart nginx`

8. Test your domain name in browser now (only http://, not https://)

If all runs smoothly, continue to [Certbot config](https://azeem-khan1.github.io/fastpages-project/week22/deploymentGuide#Certbot-Config)

## Certbot Config

> Certbot allows your site to get a certificate in order for the http request to be secure (https)

1. Run commands below

In [ ]:
cd ~

sudo snap install core; sudo snap refresh core
sudo snap install --classic certbot
sudo ln -s /snap/bin/certbot /usr/bin/certbot

sudo certbot --nginx

**Ideal outcome is shown below**

In [ ]:
Saving debug log to /var/log/letsencrypt/letsencrypt.log
Plugins selected: Authenticator nginx, Installer nginx

Which names would you like to activate HTTPS for?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1: coolcodersjava.pw
2: www.coolcodersjava.pw
3: ajarcade.duckdns.org
4: flowhealth.duckdns.org
5: goatedgroup.duckdns.org
6: jasj-inventory.duckdns.org
7: recipies.duckdns.org
8: ssvgcars.duckdns.org
9: userapi.duckdns.org
10: fr0st.ml
11: www.fr0st.ml
12: agenda.nighthawkcodescrums.gq
13: coolcoders.nighthawkcodescrums.gq
14: escaperoom.nighthawkcodescrums.gq
15: frost.nighthawkcodescrums.gq
16: jame.nighthawkcodescrums.gq
17: lawnmowers.nighthawkcodescrums.gq
18: loopholegames.nighthawkcodescrums.gq
19: musicmania.nighthawkcodescrums.gq
20: nba.nighthawkcodescrums.gq
21: sadv.nighthawkcodescrums.gq
22: ssjn.nighthawkcodescrums.gq
23: stocks.nighthawkcodescrums.gq
24: striver.nighthawkcodescrums.gq
25: tngc.nighthawkcodescrums.gq
26: white.nighthawkcodescrums.gq
27: workwatch.nighthawkcodescrums.gq
28: cars.nighthawkcodingsociety.com
29: dolphin.nighthawkcodingsociety.com
30: saakd.nighthawkcodingsociety.com
31: pythonalflask.tk
32: www.pythonalflask.tk
33: teambrobro.tk
34: www.teambrobro.tk
35: teamcheeseatimetime.tk
36: www.teamcheeseatimetime.tk
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Select the appropriate numbers separated by commas and/or spaces, or leave input
blank to select all options shown (Enter 'c' to cancel): # ENTER YOUR CORRESPONDING NUMBER

Cert not yet due for renewal

You have an existing certificate that has exactly the same domains or certificate name you requested and isn't close to expiry.
(ref: /etc/letsencrypt/renewal/nighthawkcodingsociety.com-0001.conf)

What would you like to do?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1: Attempt to reinstall this existing certificate
2: Renew & replace the cert (limit ~5 per 7 days)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Select the appropriate number [1-2] then [enter] (press 'c' to cancel): 2
Renewing an existing certificate
Performing the following challenges:
http-01 challenge for nighthawkcodingsociety.com
http-01 challenge for csa.nighthawkcodingsociety.com
http-01 challenge for cso.nighthawkcodingsociety.com
http-01 challenge for flm.nighthawkcodingsociety.com
Waiting for verification...
Cleaning up challenges
Deploying Certificate to VirtualHost /etc/nginx/sites-enabled/nighthawk_society
Deploying Certificate to VirtualHost /etc/nginx/sites-enabled/nighthawk_csa
Deploying Certificate to VirtualHost /etc/nginx/sites-enabled/nighthawk_csp
Deploying Certificate to VirtualHost /etc/nginx/sites-enabled/nighthawk_flm

Please choose whether or not to redirect HTTP traffic to HTTPS, removing HTTP access.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1: No redirect - Make no further changes to the webserver configuration.
2: Redirect - Make all requests redirect to secure HTTPS access. Choose this for
new sites, or if you're confident your site works on HTTPS. You can undo this
change by editing your web server's configuration.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Select the appropriate number [1-2] then [enter] (press 'c' to cancel): 2
Traffic on port 80 already redirecting to ssl in /etc/nginx/sites-enabled/nighthawk_society
Traffic on port 80 already redirecting to ssl in /etc/nginx/sites-enabled/nighthawk_csa
Traffic on port 80 already redirecting to ssl in /etc/nginx/sites-enabled/nighthawk_csp
Traffic on port 80 already redirecting to ssl in /etc/nginx/sites-enabled/nighthawk_flm

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Your existing certificate has been successfully renewed, and the new certificate
has been installed.

The new certificate covers the following domains:
https://nighthawkcodingsociety.com, 
https://csa.nighthawkcodingsociety.com, 
https://csp.nighthawkcodingsociety.com, and
https://flm.nighthawkcodingsociety.com,

You should test your configuration at:
https://www.ssllabs.com/ssltest/analyze.html?d=nighthawkcodingsociety.com
https://www.ssllabs.com/ssltest/analyze.html?d=csa.nighthawkcodingsociety.com
https://www.ssllabs.com/ssltest/analyze.html?d=csp.nighthawkcodingsociety.com
https://www.ssllabs.com/ssltest/analyze.html?d=flm.nighthawkcodingsociety.com
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

IMPORTANT NOTES:
 - Congratulations! Your certificate and chain have been saved at:
   /etc/letsencrypt/live/nighthawkcodingsociety.com-0001/fullchain.pem
   Your key file has been saved at:
   /etc/letsencrypt/live/nighthawkcodingsociety.com-0001/privkey.pem
   Your cert will expire on 2022-03-06. To obtain a new or tweaked
   version of this certificate in the future, simply run certbot again
   with the "certonly" option. To non-interactively renew *all* of
   your certificates, run "certbot renew"
 - If you like Certbot, please consider supporting our work by:

   Donating to ISRG / Let's Encrypt:   https://letsencrypt.org/donate
   Donating to EFF:                    https://eff.org/donate-le

## Updating after changing code in Repository

### Changing Code in VSCode

1. Make sure to `git pull` before making changes; this will make sure that you pull any changes made by your team, and prevents merge conflicts

2. Open terminal in VSCode and run `python3 main.py` (Make sure you cd into your repo)
> This should give you a local address where your flask is running. Open this in your browser to see your changes live

3. Make changes that are needed (Refer to your live site to see the changes integrated into the site)

4. Commit your changes

5. Make sure your Docker Desktop app is running

6. Test `docker-compose up` or `sudo docker-compose up` in your VSCode terminal (don't forget to 'cd' into the right repo.)

7. After it's done building, type in `http://localhost:8---` in your browser (replace '8---' with your port number you've chosen)
> Your changes should now appear

8. If all goes well, push to the repository using the Source Control tab on the left navbar

### Pulling Changes into AWS

In your AWS terminal;

1. Navigate to your repo: `cd ~/yourRepo`

2. Run `docker-compose ps` or `sudo docker-compose ps`
> It should show your docker container state as 'up' with correct port

3. `sudo docker ps` to check container and image

4. `git status` to see if you have any changes to your repo in AWS

5. `sudo docker-compose kill`
> run ps on docker-compose and docker and you should observe changes
> Server should be down (502 Bad Gateway in browser)

6. `git pull`

7. Rebuild your docker container: `sudo docker-compose up -d --build`

8. Try to curl: `curl localhost:8---` (replace '8---' with your port number)
> If your changes were visible on index.html, you should see them here

9. Test domain in browser (server should no longer be down)

10. Run `docker-compose ps` or `sudo docker-compose ps` to check that docker is up

11. `sudo docker ps` to check container and image

If all went smooth, update is complete.

## Congratulations!

Congratulations on deploying your site with AWS! A special thanks to Samit Poojary, Ethan Tran, Mr. Mortensen and Jeffrey Fonseca for helping put this together.